In [1]:
from gensim.models.keyedvectors import KeyedVectors
import numpy as np

import io
import os
import codecs
import argparse
import random

from collections import Counter
from collections import defaultdict
from itertools import product

# very useful feature used to reload python modules
from importlib import reload

# import module that loads data, tokenises the tuples, initialises the embeddings matrix
import crim_data
import semeval_data

import multiprojection_model
import multiprojection_dual
# contains code to evaluate according to semeval2018 metrics
import semeval_eval
import crim_evaluator
import crim_dual_evaluator


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# SemEval 2018, Task 9 Preamble

In [2]:
# initialise embeddings and normalise to unit-norm
#model = KeyedVectors.load_word2vec_format('embeddings/w2v_umbc_8w_10n_300d.txt', binary=False)
#model = KeyedVectors.load_word2vec_format('embeddings/glove_umbc_15w_300d.txt', binary=False)
model = KeyedVectors.load_word2vec_format('embeddings/fast_umbc_5ng_8w_300d.vec', binary=False)

#model.save_word2vec_format('embeddings/GoogleNews-vectors-negative300.txt', binary=False)
model.init_sims(replace=True)

In [3]:
# ignore words which don't feature in embeddings model
def read_subsumptions(filenames, w2v, word_type='Both'):
    hypo, hyper = filenames
        
    subsumptions = []
    is_concept = []
    
    with open(hypo, mode='r') as f_hypo, open(hyper, mode='r') as f_hyper:         
        for x, y in zip(f_hypo, f_hyper):
            query, category = x.strip().split("\t")                        
            query = query.replace(" ", "_").lower()                        
            y = y.strip()            
            # check that we have embeddings for query word            
            if (query in w2v and (word_type==category or word_type=='Both')):
                for h in y.split("\t"):
                    h = h.replace(" ", "_").lower()
                    if h in w2v:
                        subsumptions.append((query, h))
                        is_concept.append(0 if category=='Entity' else 1 )
        
        return is_concept, subsumptions
                    
# ignore vocab entries not having correspononding embeddings                                                
def read_vocab(filename, w2v):
        
    vocab = []    
    # load data itemsf
    with open(filename, mode='r') as f:        
        for word in f:
            word = word.strip().replace(" ","_").lower()            
            if word in w2v:
                vocab.append(word)
                              
    return vocab

In [4]:
# import SemEval data
data_file_names = list(map(lambda x: '1A.english.%s.data.txt'%(x), ['trial', 'test', 'training']))
gold_file_names = list(map(lambda x: '1A.english.%s.gold.txt'%(x), ['trial', 'test', 'training']))
vocab_file_name = '1A.english.vocabulary.txt'

file_names = list(zip(data_file_names, gold_file_names))
# 0 = validation; 1 = test; 2 = training
# create a dictionary for every dataset which maintains concepts, entities, and both
validation, test, training = {}, {}, {}
word_types = ['Concept', 'Entity','Both']
for w in word_types:
    validation[w] = read_subsumptions(file_names[0], model, w)[1] 
    test[w] = read_subsumptions(file_names[1], model, w)[1]
    if w == 'Both':
        is_concept, training[w] = read_subsumptions(file_names[2], model, w)
    else:
        training[w] = read_subsumptions(file_names[2], model, w)[1]

vocabulary = read_vocab(vocab_file_name, model)
    
# create hypernym dictionary
hyper_dict = defaultdict(list)
for x, y in validation['Both'] + test['Both'] + training['Both']:
    hyper_dict[x].append(y)
    
hyper_dict.default_factory = None

# print some quick stats
for w in word_types:
    print (w)
    print ("Tuples in validation set: %d; tuples in test set: %d; tuples in training: %d" 
          % (len(validation[w]), len(test[w]), len(training[w])))
    
    print ("Unique hyponyms in validation set: %d; hyponyms in test set: %d; hyponyms in training: %d"
          % (len(set([x for (x,y) in validation[w]])), len(set([x for (x,y) in test[w]])), len(set([x for (x,y) in training[w]]))))
    
    print ("Unique hypernyms in validation set: %d; hypernyms in test set: %d; hypernyms in training: %d"
          % (len(set([y for (x,y) in validation[w]])), len(set([y for (x,y) in test[w]])), len(set([y for (x,y) in training[w]]))))
    print ("-"*30)
    
print ("Vocab size: %d" % len(vocabulary))

# 647 terms were missing from the model vocab due to them appearing only once in the corpus.
# According to the technical paper, only words appearing at least 5 times were considered for vocab 
# from within the general-purpose corpus

# Also the word épée, is found without accents in the model.  Don't know why. But we'll ignore this single
# word since it should not make a difference in the scheme of things.

# total number of hyponyms
tot_hypers = len(set([y for x, y in  validation['Both'] + test['Both']  + training['Both']]))
tot_hypos = len(set([x for x, y in  validation['Both'] + test['Both']  + training['Both']]))
print ("Total number of unique hypernyms: %d; total hyponyms: %d" % (tot_hypers, tot_hypos))

Concept
Tuples in validation set: 112; tuples in test set: 4935; tuples in training: 7182
Unique hyponyms in validation set: 30; hyponyms in test set: 1057; hyponyms in training: 978
Unique hypernyms in validation set: 109; hypernyms in test set: 2574; hypernyms in training: 3475
------------------------------
Entity
Tuples in validation set: 88; tuples in test set: 2112; tuples in training: 4595
Unique hyponyms in validation set: 20; hyponyms in test set: 443; hyponyms in training: 521
Unique hypernyms in validation set: 75; hypernyms in test set: 812; hypernyms in training: 1426
------------------------------
Both
Tuples in validation set: 200; tuples in test set: 7047; tuples in training: 11777
Unique hyponyms in validation set: 50; hyponyms in test set: 1499; hyponyms in training: 1499
Unique hypernyms in validation set: 182; hypernyms in test set: 3075; hypernyms in training: 4232
------------------------------
Vocab size: 218106
Total number of unique hypernyms: 5597; total hypon

In [5]:
args = {'w2v':model,
        'train':training, 'test':test, 'validation':validation, 'vocabulary':vocabulary, 
        'is_concept':is_concept        
       }

data = semeval_data.SemevalData(args)

Initialising SemevalData...
Creating tokenizer
Dataset vocabulary size is 219034
Vocab size is 219034 words
Initialising negative sampler
Tokenising all dataset tuples
Creating embeddings matrix
Done!


In [6]:
def train_and_evaluate(hyp_model, train_split, valid_split, test_split):    
    
    # fit model
    # the test split is only used to measure the test loss
    hyp_model.fit(train_split, valid_split)    
    # this step should not be required since the model is dynamically linked to the evaluator
    hyp_model.evaluator.set_model(hyp_model.model)
    
    scores_all = []
    # evaluate trained model on word in either category separately and together
    for w in ['Concept', 'Entity', 'Both']:
        print ("Evaluating model on %s" % (w) )
        # generates predictions according to trained model
        predictions = hyp_model.evaluator.predict(test_split[w])
        # this converts the tokens back to words for evaluation
        test_tuples = data.token_to_words(test_split[w])
        # here we have a scorer that will mark our effort according to this particular test split
        scorer = semeval_eval.HypernymEvaluation(test_tuples)
        # get scores
        score_names, all_scores = scorer.get_evaluation_scores(predictions)
        # initialise scores (MRR, MAP, ...)
        scores = {s:0.0 for s in score_names }
        for k in range(len(score_names)):    
            scores[score_names[k]] = float('%.5f' % (sum([score_list[k] for score_list in all_scores]) / len(all_scores)))    
        
        scores_all.append(scores)
    return scores_all
    

# MULTI-PROJECTION Section

In [7]:
embeddings_layer = multiprojection_model.get_embeddings_model(data.embeddings_matrix, 1)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# standard model parameters - we won't be changing these
args['data']              = data
args['embeddings_layer']  = embeddings_layer
args['epochs']            = 15
args['batch_size']        = 32
args['synonym_sample_n']  = 1
args['phi_k']             = 1
args['lambda_c']          = 0.
args['negative_sample_n'] = 10
args['save_path']         = 'ft_semeval.npz'
args['patience']          = 3
args['eval_after_epoch']  = True
args['lr']                = 0.001
args['beta1']             = 0.9
args['beta2']             = 0.9
args['clip_value']        = 1.

# generate parameter combinations
#_clusters = [10, 5, 1]
_clusters = [5, 1]
#_lambda_c = [0, 0.1, 1]
_lambda_c = [0.]
#_neg_count = [10, 5, 1]
_neg_count = [10]

parameters = [_clusters, _lambda_c, _neg_count]

param_list = list(product(*parameters))

# initialise hypernymy discovery model which we will reuse by resetting the model with new args
hyp_model = multiprojection_model.MultiProjModel(args)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
# initialise final_scores dictionary
final_scores = {k:defaultdict(list) for k in param_list}

for idx2, _param in enumerate(param_list):
    print ("Running test with following parameters: phi_k: %d; lambda_c: %0.2f; neg_count: %d" \
           % (_param[0], _param[1], _param[2]))

    args['phi_k'] = _param[0]
    args['lambda_c'] = _param[1]
    args['negative_sample_n'] = _param[2]    

    for w in ['Concept', 'Entity', 'Both']:
        # train model on three sets of data
        print ("Training model on %s" % (w))
        args['save_path'] = 'ft_semeval_%d_%s.npz' % (_param[0], w)
        hyp_model.reset_model(args=args)

        all_scores = train_and_evaluate(hyp_model, 
                                        data.train_data_token[w], 
                                        data.valid_data_token[w],
                                        data.test_data_token)
        # run predictions on test

        for scores in all_scores:
            for s, v  in scores.items():
                final_scores[_param][s].append(v)

    print ("")
    print ("Finished %d from %d experiments" % (idx2+1, len(param_list)))
    print ("-"*30)

Running test with following parameters: phi_k: 5; lambda_c: 0.00; neg_count: 10
Training model on Concept
Fitting model with following parameters:
 batch_size=32;
 phi_k=5;
 lambda_c=0.00;
 epochs=15;
 negative_count=10;
 synonym_count=1
Optimizer parameters:
 lr=0.00100;
 beta1=0.900;
 beta2=0.900;
 clip=1.00
--------------------
Instructions for updating:
Use tf.cast instead.
Epoch: 1; Training Loss: 0.56345; Test Loss: 0.43088; Test MAP: 0.00000; Test MRR: 0.00000
Epoch: 2; Training Loss: 0.41159; Test Loss: 0.32091; Test MAP: 0.00000; Test MRR: 0.00000
Epoch: 3; Training Loss: 0.32432; Test Loss: 0.27120; Test MAP: 0.00000; Test MRR: 0.00000
Epoch: 4; Training Loss: 0.27479; Test Loss: 0.24108; Test MAP: 0.02311; Test MRR: 0.04444
Epoch: 5; Training Loss: 0.24261; Test Loss: 0.21384; Test MAP: 0.03141; Test MRR: 0.07500
Epoch: 6; Training Loss: 0.21873; Test Loss: 0.19162; Test MAP: 0.04435; Test MRR: 0.08333
Epoch: 7; Training Loss: 0.19988; Test Loss: 0.17810; Test MAP: 0.08423; 

Epoch: 5; Training Loss: 0.28597; Test Loss: 0.26259; Test MAP: 0.07326; Test MRR: 0.15067
Epoch: 6; Training Loss: 0.27002; Test Loss: 0.24925; Test MAP: 0.07683; Test MRR: 0.17067
Epoch: 7; Training Loss: 0.26127; Test Loss: 0.23860; Test MAP: 0.09884; Test MRR: 0.20233
Epoch: 8; Training Loss: 0.25500; Test Loss: 0.23088; Test MAP: 0.09295; Test MRR: 0.19317
Epoch: 9; Training Loss: 0.24875; Test Loss: 0.22617; Test MAP: 0.09193; Test MRR: 0.17556
Epoch: 10; Training Loss: 0.24313; Test Loss: 0.21685; Test MAP: 0.09638; Test MRR: 0.19733
Early Stop invoked at epoch 10
Done!
Evaluating model on Concept
Done 100
Done 200
Done 300
Done 400
Done 500
Done 600
Done 700
Done 800
Done 900
Done 1000
Evaluating model on Entity
Done 100
Done 200
Done 300
Done 400
Evaluating model on Both
Done 100
Done 200
Done 300
Done 400
Done 500
Done 600
Done 700
Done 800
Done 900
Done 1000
Done 1100
Done 1200
Done 1300
Done 1400

Finished 2 from 2 experiments
------------------------------


In [ ]:
final_scores

In [ ]:
for k, v in final_scores.items():    
    cl_size = k[0]
    lam = k[1]
    neg = k[2]
    for k2, v2 in v.items():        
        print ("%d\t%0.1f\t%d\t%s\t%0.5f\t%0.5f\t%0.5f" 
               % (cl_size, lam, neg, k2, v2[0], v2[1], v2[2]) )

In [10]:
for k, v in final_scores.items():    
    cl_size = k[0]
    lam = k[1]
    neg = k[2]
    for k2, v2 in v.items():        
        print ("%d\t%0.1f\t%d\t%s\t%0.5f\t%0.5f\t%0.5f\t%0.5f\t%0.5f\t%0.5f\t%0.5f\t%0.5f\t%0.5f" 
               % (cl_size, lam, neg, k2, v2[0], v2[1], v2[2], v2[3], v2[4], v2[5], v2[6], v2[7], v2[8]) )
    

5	0.0	10	MRR	0.20825	0.30513	0.23702	0.05469	0.39372	0.15492	0.21120	0.39817	0.26660
5	0.0	10	MAP	0.09915	0.13184	0.10885	0.02551	0.19507	0.07563	0.10454	0.19947	0.13268
5	0.0	10	P@1	0.16083	0.25508	0.18879	0.04257	0.33409	0.12875	0.16272	0.31603	0.20814
5	0.0	10	P@5	0.09439	0.12178	0.10255	0.02474	0.18710	0.07274	0.10057	0.19165	0.12755
5	0.0	10	P@10	0.08970	0.11577	0.09744	0.02307	0.17634	0.06838	0.09572	0.18220	0.12134
1	0.0	10	MRR	0.18564	0.23114	0.19921	0.00039	0.00266	0.00106	0.17071	0.38171	0.23312
1	0.0	10	MAP	0.09025	0.10393	0.09433	0.00023	0.00161	0.00063	0.08066	0.18969	0.11294
1	0.0	10	P@1	0.14002	0.18284	0.15277	0.00000	0.00000	0.00000	0.12961	0.32054	0.18612
1	0.0	10	P@5	0.08622	0.09703	0.08947	0.00000	0.00192	0.00057	0.07747	0.17769	0.10714
1	0.0	10	P@10	0.08253	0.09314	0.08570	0.00039	0.00189	0.00083	0.07338	0.17143	0.10239


In [ ]:
hyp_model.load_model()
#weights = np.load(hyp_model.save_path)

In [ ]:
# we score as per the usual way the predictions
predictions = {}
for w in ['Concept', 'Entity', 'Both']:
    print ("Doing %s" % (w))
    predictions[w] = hyp_model.evaluator.predict(data.test_data_token[w])        
    test_tuples = data.token_to_words(data.test_data_token[w])
    scorer = semeval_eval.HypernymEvaluation(test_tuples)
    # get scores
    score_names, all_scores = scorer.get_evaluation_scores(predictions[w])
    # initialise scores (MRR, MAP, ...)
    scores = {s:0.0 for s in score_names }
    for k in range(len(score_names)):    
        scores[score_names[k]] = float('%.5f' % (sum([score_list[k] for score_list in all_scores]) / len(all_scores)))    
    
    print (scores)


In [ ]:
plt.figure(figsize=(12,8))
ax = sns.regplot(x="Median Freq", y="MAP", data=score_freq, x_jitter=0.02, y_jitter=0.01);
ax.set(ylim=(-0.05, 1.05))

# DUAL Model Section

In [ ]:
def train_and_evaluate_dual(dual_model, train_split, valid_split, test_split):    
    
    # fit model
    # the test split is only used to measure the test loss
    dual_model.fit(train_split, valid_split)    
    # this step should not be required since the model is dynamically linked to the evaluator
    dual_model.evaluator.set_model(dual_model.feature_extractor, dual_model.concept_model, dual_model.entity_model)    
    
    scores_all = []
    # evaluate trained model on word in either category separately and together
    for w in ['Concept', 'Entity', 'Both']:
        print ("Evaluating model on %s" % (w) )
        # generates predictions according to trained model
        predictions = dual_model.evaluator.predict(test_split[w])
        # this converts the tokens back to words for evaluation
        test_tuples = data.token_to_words(test_split[w])
        # here we have a scorer that will mark our effort according to this particular test split
        scorer = semeval_eval.HypernymEvaluation(test_tuples)
        # get scores
        score_names, all_scores = scorer.get_evaluation_scores(predictions)
        # initialise scores (MRR, MAP, ...)
        scores = {s:0.0 for s in score_names }
        for k in range(len(score_names)):    
            scores[score_names[k]] = float('%.5f' % (sum([score_list[k] for score_list in all_scores]) / len(all_scores)))    
        
        scores_all.append(scores)                    
    return scores_all

def evaluate_only_dual(dual_model, test_split):
    predictions = dual_model.evaluator.predict(test_split)
    # this converts the tokens back to words for evaluation
    test_tuples = data.token_to_words(test_split)
    # here we have a scorer that will mark our effort according to this particular test split
    scorer = semeval_eval.HypernymEvaluation(test_tuples)
    # get scores
    score_names, all_scores = scorer.get_evaluation_scores(predictions)
    # initialise scores (MRR, MAP, ...)
    scores = {s:0.0 for s in score_names }
    for k in range(len(score_names)):    
        scores[score_names[k]] = float('%.5f' % (sum([score_list[k] for score_list in all_scores]) / len(all_scores)))    

    return predictions, scores

In [ ]:
embeddings_layer = multiprojection_model.get_embeddings_model(data.embeddings_matrix, 1)

In [ ]:
# standard model parameters - we won't be changing these
args['data']              = data
args['embeddings_layer']  = embeddings_layer
args['epochs']            = 20
args['batch_size']        = 64
args['synonym_sample_n']  = 1
args['phi_k']             = 1
args['lambda_c']          = 0.
args['negative_sample_n'] = 10
args['save_path']         = 'dual_ft_semeval.npz'
args['patience']          = 5
args['eval_after_epoch']  = True
args['lr']                = 0.001
args['beta1']             = 0.9
args['beta2']             = 0.9
args['clip_value']        = 1.


dual_model = multiprojection_dual.MultiProjModelDual(args)

# generate parameter combinations
#_clusters = [10, 5, 1]
_clusters = [1, 5, 10]
#_lambda_c = [0, 0.1, 1]
_lambda_c = [0.]
#_neg_count = [10, 5, 1]
_neg_count = [10]

parameters = [_clusters, _lambda_c, _neg_count]

param_list = list(product(*parameters))


In [ ]:
# initialise final_scores dictionary
final_scores = {k:defaultdict(list) for k in param_list}

for idx2, _param in enumerate(param_list):
    print ("Running test with following parameters: phi_k: %d; lambda_c: %0.2f; neg_count: %d" \
           % (_param[0], _param[1], _param[2]))

    args['phi_k'] = _param[0]
    args['lambda_c'] = _param[1]
    args['negative_sample_n'] = _param[2]    

    for w in ['Both']:    
        # train model on three sets of data
        print ("Training model on %s" % (w))
        args['save_path'] = 'dual_ft_semeval_%s.npz' % (w)
        dual_model.reset_model(args=args)

        all_scores = train_and_evaluate_dual(dual_model, 
                                             data.train_data_token[w], 
                                             data.valid_data_token[w],
                                             data.test_data_token)
        # run predictions on test

        for scores in all_scores:
            for s, v  in scores.items():
                final_scores[_param][s].append(v)

    print ("")
    print ("Finished %d from %d experiments" % (idx2+1, len(param_list)))
    print ("-"*30)

    

In [ ]:
for k, v in final_scores.items():    
    cl_size = k[0]
    lam = k[1]
    neg = k[2]
    for k2, v2 in v.items():        
        print ("%d\t%0.1f\t%d\t%s\t%0.5f\t%0.5f\t%0.5f" 
               % (cl_size, lam, neg, k2, v2[0], v2[1], v2[2]) )

# Analysis of Prediction quality vs hypernym freq in training set

In [ ]:
# perform freq analysis of all hypernyms in training set; query terms will not features in test but test terms
# will certainly be related to hypernyms found in training set.
def get_hypernym_freq(dataset):
    all_hypernyms = Counter([y for x, y in data.token_to_words(dataset)])
    cnt_distinct_hyper = sum(all_hypernyms.values())
    #sorted([(y, x) for x, y in all_hypernyms.items()], reverse=True)
    hyper_freq = {w:round((cnt/cnt_distinct_hyper), 5) for w, cnt in all_hypernyms.items()}
    return hyper_freq

In [ ]:
np.sum(list(hyper_freq.values()))

In [ ]:
predictions = yummy.evaluator.predict(test_data_split[4])

In [ ]:
predictions

In [ ]:
# we score as per the usual way the predictions
test_tuples = data.token_to_words(test_data_split[4])
scorer = semeval_eval.HypernymEvaluation(test_tuples)
# get scores
score_names, all_scores = scorer.get_evaluation_scores(predictions)
# initialise scores (MRR, MAP, ...)
scores = {s:0.0 for s in score_names }
for k in range(len(score_names)):    
    scores[score_names[k]] = float('%.5f' % (sum([score_list[k] for score_list in all_scores]) / len(all_scores)))    
    
# we create a dictionary of ground truth hypernyms for the test split of interest    
ground_truth = defaultdict(list)
for x, y in test_tuples:
    ground_truth[x].append(y)
ground_truth.default_factory = None    

In [ ]:
# all MAP scores
list(predictions.items())[34]

In [ ]:
def get_score_freq_matrix(test_data, predictions, hyper_freq, jitter=False):
    # we score as per the usual way the predictions
    test_tuples = data.token_to_words(test_data)
    scorer = semeval_eval.HypernymEvaluation(test_tuples)
    # get scores
    score_names, all_scores = scorer.get_evaluation_scores(predictions)
    # initialise scores (MRR, MAP, ...)
    scores = {s:0.0 for s in score_names }
    for k in range(len(score_names)):    
        scores[score_names[k]] = float('%.5f' % (sum([score_list[k] for score_list in all_scores]) / len(all_scores)))    

    # we create a dictionary of ground truth hypernyms for the test split of interest    
    ground_truth = defaultdict(list)
    for x, y in test_tuples:
        ground_truth[x].append(y)
    ground_truth.default_factory = None    


    # retain MAP scores only from score list of lists
    all_map = np.round(np.asarray(all_scores)[:,1], 3)

    # iterate over every query term in test set; create dataset with AP score for word, median freq of ground
    # truth hypernym, based on appearance in training set.  Hypernyms that did not appear at all, will be assigned
    # freq of 0.
    score_freq_matrix = np.zeros((all_map.shape[0], 2))
    # sort prediction keys explicityly to make sure we process in the same order processed by evaluator
    for idx, w in enumerate(sorted(predictions.keys())):
        # find freq of predictions
        score_freq_matrix[idx][0] = all_map[idx]
        gold = ground_truth[w]
        score_freq_matrix[idx][1] = np.median(([hyper_freq[g] if g in hyper_freq else 0. for g in gold]))
        
    if jitter:
        # add some jitter to the signal to make it easier to interpret in the scatterplot
        mu, sigma = 0, 0.01 
        # creating a noise with the same dimension as the dataset (2,2) 
        noise = np.random.normal(mu, sigma, (all_map.shape[0], 2) )
        score_freq_matrix =  score_freq_matrix + noise

    score_freq = pd.DataFrame(score_freq_matrix, columns=['AP', 'Median Freq'])
    # add query word to data frame
    score_freq = score_freq.assign(word=pd.Series(list(predictions.keys())).values)
    return score_freq

In [ ]:
#score_freq.loc[score_freq.MAP == 1.,].sort_values('Median Freq', ascending=False)

# prediction contains the generated hypernyms for YAMANE on the 5th fold of the training_data
score_freq_yam=get_score_freq_matrix(test_data_split[4], predictions, hyper_freq)

# prediction_2 contains the generated hypernyms for CRIM on the 5th fold of the training data
score_freq=get_score_freq_matrix(test_data_split[4], predictions_2, hyper_freq)

In [ ]:
#sns.relplot(x="Median Freq", y="MAP", data=score_freq);
plt.figure(figsize=(12,8))
ax = sns.regplot(x="Median Freq", y="AP", data=score_freq_yam, x_jitter=0.01, y_jitter=0.005, marker="x");
ax.set(ylim=(-0.05, 1.05))

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.regplot(x="Median Freq", y="AP", data=score_freq, x_jitter=0.01, y_jitter=0.005);
ax.set(ylim=(-0.05, 1.05))


In [ ]:
top_yummy_wrong

In [ ]:
# analyze top-ranked word for term words scoring 0 MAP
yummy_wrong_terms = score_freq_yam.loc[score_freq_yam.MAP==0, 'word'].tolist()
crim_wrong_terms  = score_freq.loc[score_freq.MAP==0, 'word'].tolist()

top_crim_wrong = []
top_yummy_wrong = []

for w in crim_wrong_terms:
    top_crim_wrong.append(predictions_2[w][0])
    
for w in yummy_wrong_terms:
    top_yummy_wrong.append(predictions[w][0])    
    
top_crim_wrong =  sorted([(v,k) for k, v in Counter(top_crim_wrong).items()], reverse=True)[:15]
top_yummy_wrong =  sorted([(v,k) for k, v in Counter(top_yummy_wrong).items()], reverse=True)[:15]

# multiply words according to frequency
yummy_wrong_flat = [li for lol in list(map(lambda w: [w[1]] * w[0], top_yummy_wrong)) for li in lol  ]
crim_wrong_flat = [li for lol in list(map(lambda w: [w[1]] * w[0], top_crim_wrong)) for li in lol  ]

combined_wrong_list = list(zip(['CRIM'] * len(crim_wrong_flat), crim_wrong_flat))
combined_wrong_list.extend(list(zip(['Yamane'] * len(yummy_wrong_flat), yummy_wrong_flat)))

In [ ]:
incorrect_df =  pd.DataFrame(combined_wrong_list, columns=['Model', 'Highest Ranked Incorrect Word'])

In [ ]:
incorrect_df.loc[incorrect_df.Model == 'CRIM',]

In [ ]:
plt.figure(figsize=(12,8))
g = sns.countplot(x="Highest Ranked Incorrect Word", 
                  palette=sns.cubehelix_palette(15, start=2, rot=0.35, dark=0.47, light=0.85, reverse=True), 
                  data=incorrect_df.loc[incorrect_df.Model == 'CRIM',])
g.set_xticklabels(g.get_xticklabels(), rotation=30)

In [ ]:
for w in sorted([(v, k)for k, v in hyper_freq.items()], reverse=True)[:10]:
    print (w[1], w[0])
    
hyper_freq['action']

In [ ]:
plt.figure(figsize=(12,8))
g = sns.countplot(x="Highest Ranked Incorrect Word", 
                  palette=sns.cubehelix_palette(15, start=2, rot=0.35, dark=0.47, light=0.85, reverse=True), 
                  data=incorrect_df.loc[incorrect_df.Model == 'Yamane',])
g.set_xticklabels(g.get_xticklabels(), rotation=30)

In [ ]:
for w in sorted([(v, k)for k, v in hyper_freq.items()], reverse=True)[:10]:
    print (w[1], w[0])

In [ ]:
# least frequent hypernyms
for w in [(v, k) for k, v in hyper_freq.items() if v == min(hyper_freq.values())][:10]:
    print (w[1], w[0])

In [ ]:
# let's see good scores low frequency
good_words = score_freq.loc[(score_freq.AP >= 1.0) & (score_freq['Median Freq'] < 0.005) , 'word'].tolist()
score_freq.loc[(score_freq.AP >= 1.0) & (score_freq['Median Freq'] < 0.005) , ]

In [ ]:
for w in good_words:
    print ("%s: %s" % (w, ", ".join(predictions_2[w])))
    print ("%s: %s" % (w, ", ".join(hyper_dict[w])))
    print ("-"*30)
    
#print (hyper_dict['intercourse'])
#predictions_2['intercourse']


# Scratch Pad

In [ ]:
reload(semeval_data)
reload(multiprojection_model)
reload(crim_evaluator)
reload(multiprojection_dual)
reload(crim_dual_evaluator)

In [ ]:
len(set(read_vocab(vocab_file_name)).difference(set(vocabulary)))

In [ ]:
# remove vocab term having no vector in embeddings
def get_terms_having_vectors(w2v, dataset):
    return [(q,h) for q, h in dataset if q in w2v and h in w2v]

# remove any words which don't have corresponding embeddings 
for w in word_types:
    validation[w] = get_terms_having_vectors(model, validation[w])
    test[w] = get_terms_having_vectors(model, test[w])
    training[w] = get_terms_having_vectors(model, training[w])

vocabulary = list(filter(lambda w: w in model, vocabulary))   